In [ ]:
import brainpy.respa as respa
import brainpy as bp
a = respa.LIF(3200,V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5., method='exp_auto', V_initializer=bp.initialize.Normal(-55., 2.))
b = respa.LIF(800,V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5., method='exp_auto', V_initializer=bp.initialize.Normal(-55., 2.))
c = respa.Exponential(a, b, bp.conn.FixedProb(0.02), g_max=10,tau=5., output=bp.synouts.COBA(E=0.), method='exp_auto')

net = respa.Network()
net.build()
runner = respa.DSRunner(
    net,
    monitors={'spikes': b.spike},
    inputs=[(a.input, 20.)]
)
runner.run(100.)
print(net.lowref.nodes())
bp.visualize.raster_plot(runner.mon.ts, runner.mon['spikes'], show=True)



In [ ]:
print(net.lowref.nodes())
runner.run(100.)
bp.visualize.raster_plot(runner.mon.ts, runner.mon['spikes'], show=True)